Need to download agency_data.csv file first.

2017-Q1 doesn't have any data, so the content should be null.

In [ ]:
import requests
import json
import os
import pandas as pd

BASE_URL = "https://api.usaspending.gov/api/v2/spending/"
HEADERS = {
    "Content-Type": "application/json"
}

# 定义财年和季度
fiscal_years = range(2017, 2020)  # 2017-2019
quarters = {
    "Q1": 1,
    "Q2": 2,
    "Q3": 3,
    "Q4": 4
}

OUTPUT_DIR = "federal_account_json"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def fetch_federal_account_data(agency_id, fiscal_year, quarter):
    payload = {
        "type": "federal_account",
        "filters": {
            "fy": fiscal_year,
            "quarter": str(quarter),
            "agency": agency_id
        }
    }
    response = requests.post(BASE_URL, json=payload, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("results", [])
    else:
        print(f"Failed to fetch data for Agency {agency_id}, FY {fiscal_year}, Quarter {quarter}. "
              f"Status code: {response.status_code}, Response: {response.text}")
        return []

agency_data = pd.read_csv('agency_data.csv') 
agency_ids = agency_data['Agency_ID'] 
agency_ids = agency_ids.iloc[1:]
agency_ids = agency_ids.astype(int)
agency_ids = agency_ids.tolist()

for fiscal_year in fiscal_years:
    for quarter_label, quarter in quarters.items():
        for agency_id in agency_ids:
            print(f"Processing data for Agency {agency_id}, FY {fiscal_year}, Quarter {quarter_label}...")

            # obtain data
            results = fetch_federal_account_data(agency_id, fiscal_year, quarter)

            # JSON
            json_output = {
                "agency_id": agency_id,
                "federal_ids": []
            }
            total_amount = sum(item.get("amount", 0.0) for item in results)
            for item in results:
                federal_id_data = {
                    "federal_id": item.get("id"),
                    "federal_account_name": item.get("name"),
                    "total_amount": item.get("amount", 0.0),
                    "percent": f"{(item.get('amount', 0.0) / total_amount) * 100:.2f}%" if total_amount else "0%"
                }
                json_output["federal_ids"].append(federal_id_data)

            # Save JSON file
            file_name = f"agency_{agency_id}_federal_ids_{fiscal_year}_{quarter_label}.json"
            file_path = os.path.join(OUTPUT_DIR, file_name)
            with open(file_path, "w") as json_file:
                json.dump(json_output, json_file, indent=4)

            print(f"Saved JSON file for Agency {agency_id}, FY {fiscal_year}, Quarter {quarter_label} at {file_path}")
